In [1]:
"""
Notebook that runs grid searches for different neural network sizes to find best hyperparameters
with option prices generated by both Merton Jump Diffusion and NIG models,
then trains the models with those parameters and evaluates the performance on unseen data
"""


import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from tensorflow import keras
from tensorflow.random import set_seed
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from math import floor

data_path = 'C:\\Users\\oli-w\\OneDrive\\Uni\\Master Thesis\\Neural-Network-Option-Pricing-Master-Thesis\\Code\\OptionData\\'
result_path = 'C:\\Users\\oli-w\\OneDrive\\Uni\\Master Thesis\\Neural-Network-Option-Pricing-Master-Thesis\\Code\\Results\\'

np.random.seed(seed=123)
set_seed(1234)

# 1 Prepare Data

In [2]:
T = [0.3, 1, 3]

In [3]:
def get_train_test_data(model_name):
    df = pd.read_csv(data_path + 'one_dim_{}_prices.csv'.format(model_name))
    X = np.array(df['s']).reshape(len(df), 1)
    y = np.array(df[['0.3', '1.0', '3.0']])
    X_train, X_test, y_train_all, y_test_all = train_test_split(X, y, test_size=0.2)
    y_train_dict, y_test_dict = [{0.3: y_arr.T[0], 1: y_arr.T[1], 3: y_arr.T[2]} for y_arr in (y_train_all, y_test_all)]
    return X_train, X_test, y_train_dict, y_test_dict

In [4]:
"""
Merton model data
"""

X_train_mert, X_test_mert, y_train_mert, y_test_mert = get_train_test_data('merton')

In [5]:
"""
NIG model data
"""

X_train_nig, X_test_nig, y_train_nig, y_test_nig = get_train_test_data('nig')

# 2 Grid Search

## 2.1 Helper Functions

In [6]:
"""
Helper function to create model from given number of weights and hidden layers
"""

def create_model(hidden_layers=1, learning_rate=0.001, n_weights=10):
    # formulas derived from nWeights = sum (d(l-1)+1)*d(l) for all layers l with output dim d(l)
    
    if hidden_layers == 1: # 100% of neurons in first hidden layer
        neurons = [floor((n_weights - 1) / 3)]
    elif hidden_layers == 2: # 70% / 30% split of neurons
        x = 1/7 * (np.sqrt(21 * n_weights + 79) - 10)
        neurons = list(map(floor,[7/3 * x, x]))
    elif hidden_layers == 3: # 50% / 30% / 20% split
        x = 1/21 * (np.sqrt(84 * n_weights + 205) - 17)
        neurons = list(map(floor, [5/2 * x, 3/2 * x, x]))
    else:
        raise Exception('Only 1, 2 or 3 layers allowed')
        
    model = Sequential([Dense(neurons[0], activation='relu', input_dim=1)])
    for n in neurons[1:]:
        model.add(Dense(n, activation='relu'))
    model.add(Dense(1))

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), loss='mse')

    return model


In [7]:
"""
Runs grid search for given train and test data with network of given size
"""

def run_grid_search(X_train, y_train, n_weights, n_epochs):
    
    batch_size = [128, 256, 512, 1024]
    learning_rate = [0.1, 0.01, 0.001, 0.0001]
    hidden_layers = [1, 2, 3]

    p_grid = dict(hidden_layers=hidden_layers, batch_size=batch_size, learning_rate=learning_rate, epochs=n_epochs)
    
    def creator(hidden_layers, learning_rate): return create_model(hidden_layers, learning_rate, n_weights)
    
    model = KerasRegressor(creator, verbose=0)
    grid = GridSearchCV(estimator=model, param_grid=p_grid, n_jobs=-1, cv=4, verbose=3)
    grid_result = grid.fit(X_train, y_train)
    print('Best: {} using {}'.format(grid_result.best_score_, grid_result.best_params_))
    return grid_result

In [8]:
"""
Saves performance result for all parameters and best results for each network size
"""

def save_results(result_dict, pricer_name, maturity):

    columns = ['mean_fit_time', 'param_batch_size', 'param_hidden_layers', 'param_learning_rate', 
                'param_epochs', 'mean_test_score', 'rank_test_score']
    dfs = []
    best_dfs = []
    for n in result_dict.keys():
        df = pd.DataFrame(result_dict[n].cv_results_)[columns]
        df['n_weights'] = n
        df.columns = ['Training Time (s)', 'Batch Size', 'Hidden Layers', 'Learning Rate', 'Epochs',
                      'Validation MSE', 'Rank', 'Weights']
        df = df[['Weights', 'Validation MSE', 'Hidden Layers', 'Learning Rate', 'Batch Size',
                 'Epochs', 'Training Time (s)', 'Rank']]
        df['Validation MSE'] = (-df['Validation MSE']).apply(lambda x: '{:.3e}'.format(x))
        dfs.append(df)
        best_dfs.append(df[df['Rank'] == 1].drop(['Rank'], axis=1))
    result_df = pd.concat(dfs).reset_index(drop=True)
    best_result_df = pd.concat(best_dfs).reset_index(drop=True)
    result_df.to_csv(result_path + 'one_dim_{}_{}_grid_search_all_results.csv'.format(pricer_name, maturity), index=False)
    best_result_df.to_csv(result_path + 'one_dim_{}_{}_grid_search_overview.csv'.format(pricer_name, maturity), index=False)
    return best_result_df

## 2.1 Perform Grid Search

In [9]:
n_epochs = {10: [10, 25, 50, 100],
               30: [10, 25, 50, 100],
               60: [10, 25, 50, 100],
               100: [10, 25, 50, 100],
               300: [20, 50, 100, 150],
               600: [20, 50, 100, 150],
               1000: [20, 50, 100, 150],
               3000: [50, 100, 150, 200],
               6000: [50, 100, 150, 200],
               10000: [50, 100, 175, 300],
               30000: [100, 175, 300, 500]}

n_weights = n_epochs.keys()

In [10]:
"""
Merton Prices
"""

res_mert = {}
for t in T:
    print('T = {}'.format(t))
    res_mert[t] = {}
    for n in n_weights:
        print('Network Size = {}'.format(n))
        res_mert[t][n] = run_grid_search(X_train_mert, y_train_mert[t], n, n_epochs[n])
    res_mert[t] = save_results(res_mert[t], 'merton', t)
    display(res_mert[t])

T = 0.3
Network Size = 10
Fitting 4 folds for each of 192 candidates, totalling 768 fits
Best: -6.955959679544321e-05 using {'batch_size': 256, 'epochs': 50, 'hidden_layers': 1, 'learning_rate': 0.1}
Network Size = 30
Fitting 4 folds for each of 192 candidates, totalling 768 fits
Best: -1.2293572012822551e-05 using {'batch_size': 128, 'epochs': 25, 'hidden_layers': 1, 'learning_rate': 0.1}
Network Size = 60
Fitting 4 folds for each of 192 candidates, totalling 768 fits
Best: -6.0746091321561835e-06 using {'batch_size': 128, 'epochs': 25, 'hidden_layers': 1, 'learning_rate': 0.01}
Network Size = 100
Fitting 4 folds for each of 192 candidates, totalling 768 fits
Best: -2.7290861908113584e-06 using {'batch_size': 512, 'epochs': 50, 'hidden_layers': 1, 'learning_rate': 0.01}
Network Size = 300
Fitting 4 folds for each of 192 candidates, totalling 768 fits
Best: -1.2460496545685373e-07 using {'batch_size': 128, 'epochs': 150, 'hidden_layers': 1, 'learning_rate': 0.01}
Network Size = 600
Fit

,Weights,Validation MSE,Hidden Layers,Learning Rate,Batch Size,Epochs,Training Time (s)
0,10,6.956e-05,1,0.1,256,50,2.451195
1,30,1.229e-05,1,0.1,128,25,2.600796
2,60,6.075e-06,1,0.01,128,25,3.285465
3,100,2.729e-06,1,0.01,512,50,2.387366
4,300,1.246e-07,1,0.01,128,150,15.565377
5,600,4.722e-08,1,0.01,128,20,3.245072
6,1000,8.389e-09,1,0.01,256,50,5.473862
7,3000,1.021e-09,1,0.01,512,150,26.944199
8,6000,1.445e-09,1,0.001,512,200,85.235075
9,10000,9.920e-10,1,0.0001,256,300,174.480709


T = 1
Network Size = 10
Fitting 4 folds for each of 192 candidates, totalling 768 fits
Best: -9.496031861999654e-05 using {'batch_size': 512, 'epochs': 50, 'hidden_layers': 1, 'learning_rate': 0.1}
Network Size = 30
Fitting 4 folds for each of 192 candidates, totalling 768 fits


C:\Users\oli-w\anaconda3\envs\tensorflow\lib\site-packages\joblib\externals\loky\process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best: -1.5984647802724794e-05 using {'batch_size': 256, 'epochs': 50, 'hidden_layers': 1, 'learning_rate': 0.1}
Network Size = 60
Fitting 4 folds for each of 192 candidates, totalling 768 fits
Best: -9.384470558870817e-06 using {'batch_size': 256, 'epochs': 50, 'hidden_layers': 1, 'learning_rate': 0.1}
Network Size = 100
Fitting 4 folds for each of 192 candidates, totalling 768 fits
Best: -1.6817675998481718e-06 using {'batch_size': 512, 'epochs': 100, 'hidden_layers': 1, 'learning_rate': 0.01}
Network Size = 300
Fitting 4 folds for each of 192 candidates, totalling 768 fits
Best: -1.2820291761528324e-07 using {'batch_size': 256, 'epochs': 100, 'hidden_layers': 1, 'learning_rate': 0.01}
Network Size = 600
Fitting 4 folds for each of 192 candidates, totalling 768 fits
Best: -3.408283522077227e-08 using {'batch_size': 512, 'epochs': 150, 'hidden_layers': 1, 'learning_rate': 0.01}
Network Size = 1000
Fitting 4 folds for each of 192 candidates, totalling 768 fits
Best: -3.636673273632596e-

,Weights,Validation MSE,Hidden Layers,Learning Rate,Batch Size,Epochs,Training Time (s)
0,10,9.496e-05,1,0.1,512,50,2.758873
1,30,1.598e-05,1,0.1,256,50,3.053585
2,60,9.384e-06,1,0.1,256,50,3.260282
3,100,1.682e-06,1,0.01,512,100,3.820533
4,300,1.282e-07,1,0.01,256,100,6.693850
5,600,3.408e-08,1,0.01,512,150,8.150704
6,1000,3.637e-08,1,0.001,256,150,12.980290
7,3000,5.206e-09,1,0.01,512,200,34.059174
8,6000,4.898e-09,1,0.0001,128,200,58.622240
9,10000,3.353e-09,1,0.0001,128,175,91.729210


T = 3
Network Size = 10
Fitting 4 folds for each of 192 candidates, totalling 768 fits
Best: -0.0004462307424546452 using {'batch_size': 256, 'epochs': 25, 'hidden_layers': 1, 'learning_rate': 0.1}
Network Size = 30
Fitting 4 folds for each of 192 candidates, totalling 768 fits
Best: -3.352268004164216e-05 using {'batch_size': 512, 'epochs': 25, 'hidden_layers': 1, 'learning_rate': 0.1}
Network Size = 60
Fitting 4 folds for each of 192 candidates, totalling 768 fits
Best: -1.1217867381674296e-05 using {'batch_size': 512, 'epochs': 100, 'hidden_layers': 1, 'learning_rate': 0.1}
Network Size = 100
Fitting 4 folds for each of 192 candidates, totalling 768 fits
Best: -3.42421435561846e-06 using {'batch_size': 256, 'epochs': 50, 'hidden_layers': 1, 'learning_rate': 0.01}
Network Size = 300
Fitting 4 folds for each of 192 candidates, totalling 768 fits


C:\Users\oli-w\anaconda3\envs\tensorflow\lib\site-packages\joblib\externals\loky\process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best: -1.2400022075809147e-07 using {'batch_size': 128, 'epochs': 150, 'hidden_layers': 1, 'learning_rate': 0.01}
Network Size = 600
Fitting 4 folds for each of 192 candidates, totalling 768 fits
Best: -1.7827717169183188e-07 using {'batch_size': 256, 'epochs': 100, 'hidden_layers': 1, 'learning_rate': 0.01}
Network Size = 1000
Fitting 4 folds for each of 192 candidates, totalling 768 fits
Best: -6.87489460915458e-08 using {'batch_size': 256, 'epochs': 100, 'hidden_layers': 1, 'learning_rate': 0.01}
Network Size = 3000
Fitting 4 folds for each of 192 candidates, totalling 768 fits
Best: -3.3883853778338135e-09 using {'batch_size': 512, 'epochs': 200, 'hidden_layers': 1, 'learning_rate': 0.01}
Network Size = 6000
Fitting 4 folds for each of 192 candidates, totalling 768 fits
Best: -3.905075346422038e-09 using {'batch_size': 512, 'epochs': 200, 'hidden_layers': 1, 'learning_rate': 0.01}
Network Size = 10000
Fitting 4 folds for each of 192 candidates, totalling 768 fits
Best: -7.920869862

,Weights,Validation MSE,Hidden Layers,Learning Rate,Batch Size,Epochs,Training Time (s)
0,10,4.462e-04,1,0.1,256,25,2.282397
1,30,3.352e-05,1,0.1,512,25,1.732367
2,60,1.122e-05,1,0.1,512,100,3.572697
3,100,3.424e-06,1,0.01,256,50,4.786949
4,300,1.240e-07,1,0.01,128,150,23.317149
5,600,1.783e-07,1,0.01,256,100,9.513062
6,1000,6.875e-08,1,0.01,256,100,10.408917
7,3000,3.388e-09,1,0.01,512,200,40.398721
8,6000,3.905e-09,1,0.01,512,200,81.257230
9,10000,7.921e-11,1,0.01,512,175,111.887337


In [10]:
"""
NIG Prices
"""

np.random.seed(seed=123)
set_seed(1234)

res_nig = {}
for t in T:
    print('T = {}'.format(t))
    res_nig[t] = {}
    for n in n_weights:
        print('Network Size = {}'.format(n))
        res_nig[t][n] = run_grid_search(X_train_nig, y_train_nig[t], n, n_epochs[n])
    res_nig[t] = save_results(res_nig[t], 'nig', t)
    display(res_nig[t])

T = 0.3
Network Size = 10
Fitting 4 folds for each of 192 candidates, totalling 768 fits
Best: -8.507341954100411e-05 using {'batch_size': 256, 'epochs': 100, 'hidden_layers': 1, 'learning_rate': 0.1}
Network Size = 30
Fitting 4 folds for each of 192 candidates, totalling 768 fits
Best: -3.165220414302894e-05 using {'batch_size': 128, 'epochs': 50, 'hidden_layers': 1, 'learning_rate': 0.01}
Network Size = 60
Fitting 4 folds for each of 192 candidates, totalling 768 fits
Best: -6.059807503788761e-06 using {'batch_size': 512, 'epochs': 50, 'hidden_layers': 1, 'learning_rate': 0.1}
Network Size = 100
Fitting 4 folds for each of 192 candidates, totalling 768 fits
Best: -2.982568830134369e-06 using {'batch_size': 128, 'epochs': 25, 'hidden_layers': 1, 'learning_rate': 0.01}
Network Size = 300
Fitting 4 folds for each of 192 candidates, totalling 768 fits
Best: -1.901968751383265e-07 using {'batch_size': 512, 'epochs': 50, 'hidden_layers': 1, 'learning_rate': 0.01}
Network Size = 600
Fitting

,Weights,Validation MSE,Hidden Layers,Learning Rate,Batch Size,Epochs,Training Time (s)
0,10,8.507e-05,1,0.1,256,100,4.805398
1,30,3.165e-05,1,0.01,128,50,5.518491
2,60,6.060e-06,1,0.1,512,50,2.646672
3,100,2.983e-06,1,0.01,128,25,3.087513
4,300,1.902e-07,1,0.01,512,50,2.959336
5,600,2.637e-08,1,0.01,1024,100,4.344380
6,1000,5.793e-09,1,0.01,256,100,9.729728
7,3000,1.551e-09,1,0.001,512,200,34.668279
8,6000,6.552e-10,1,0.001,512,150,55.889272
9,10000,3.335e-10,1,0.001,1024,300,168.226073


T = 1
Network Size = 10
Fitting 4 folds for each of 192 candidates, totalling 768 fits
Best: -0.000171375485479075 using {'batch_size': 256, 'epochs': 100, 'hidden_layers': 1, 'learning_rate': 0.1}
Network Size = 30
Fitting 4 folds for each of 192 candidates, totalling 768 fits
Best: -4.083521980646765e-05 using {'batch_size': 128, 'epochs': 10, 'hidden_layers': 1, 'learning_rate': 0.1}
Network Size = 60
Fitting 4 folds for each of 192 candidates, totalling 768 fits


C:\Users\oli-w\anaconda3\envs\tensorflow\lib\site-packages\joblib\externals\loky\process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best: -1.1309656429148163e-05 using {'batch_size': 1024, 'epochs': 100, 'hidden_layers': 1, 'learning_rate': 0.1}
Network Size = 100
Fitting 4 folds for each of 192 candidates, totalling 768 fits
Best: -3.746074128230248e-06 using {'batch_size': 512, 'epochs': 100, 'hidden_layers': 1, 'learning_rate': 0.01}
Network Size = 300
Fitting 4 folds for each of 192 candidates, totalling 768 fits
Best: -1.2530992776760286e-07 using {'batch_size': 128, 'epochs': 150, 'hidden_layers': 1, 'learning_rate': 0.01}
Network Size = 600
Fitting 4 folds for each of 192 candidates, totalling 768 fits
Best: -2.9799449352552188e-08 using {'batch_size': 256, 'epochs': 50, 'hidden_layers': 1, 'learning_rate': 0.01}
Network Size = 1000
Fitting 4 folds for each of 192 candidates, totalling 768 fits
Best: -1.4986132557126552e-08 using {'batch_size': 512, 'epochs': 100, 'hidden_layers': 1, 'learning_rate': 0.01}
Network Size = 3000
Fitting 4 folds for each of 192 candidates, totalling 768 fits
Best: -4.75581479753

,Weights,Validation MSE,Hidden Layers,Learning Rate,Batch Size,Epochs,Training Time (s)
0,10,1.714e-04,1,0.1,256,100,4.945274
1,30,4.084e-05,1,0.1,128,10,1.927095
2,60,1.131e-05,1,0.1,1024,100,2.583590
3,100,3.746e-06,1,0.01,512,100,3.468973
4,300,1.253e-07,1,0.01,128,150,17.766732
5,600,2.980e-08,1,0.01,256,50,4.390508
6,1000,1.499e-08,1,0.01,512,100,7.706140
7,3000,4.756e-09,1,0.01,512,200,34.509703
8,6000,1.360e-09,1,0.01,512,150,56.329594
9,10000,3.925e-09,1,0.01,512,300,188.843061


T = 3
Network Size = 10
Fitting 4 folds for each of 192 candidates, totalling 768 fits


C:\Users\oli-w\anaconda3\envs\tensorflow\lib\site-packages\joblib\externals\loky\process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best: -0.0003248741031711688 using {'batch_size': 128, 'epochs': 100, 'hidden_layers': 1, 'learning_rate': 0.1}
Network Size = 30
Fitting 4 folds for each of 192 candidates, totalling 768 fits
Best: -3.400500088446279e-05 using {'batch_size': 256, 'epochs': 100, 'hidden_layers': 1, 'learning_rate': 0.1}
Network Size = 60
Fitting 4 folds for each of 192 candidates, totalling 768 fits
Best: -7.643980097782332e-06 using {'batch_size': 512, 'epochs': 100, 'hidden_layers': 1, 'learning_rate': 0.1}
Network Size = 100
Fitting 4 folds for each of 192 candidates, totalling 768 fits
Best: -3.943480407997413e-06 using {'batch_size': 256, 'epochs': 100, 'hidden_layers': 1, 'learning_rate': 0.01}
Network Size = 300
Fitting 4 folds for each of 192 candidates, totalling 768 fits
Best: -6.047910883211216e-08 using {'batch_size': 128, 'epochs': 150, 'hidden_layers': 1, 'learning_rate': 0.01}
Network Size = 600
Fitting 4 folds for each of 192 candidates, totalling 768 fits
Best: -5.555560234071777e-08 u

,Weights,Validation MSE,Hidden Layers,Learning Rate,Batch Size,Epochs,Training Time (s)
0,10,3.249e-04,1,0.1,128,100,9.089689
1,30,3.401e-05,1,0.1,256,100,6.153043
2,60,7.644e-06,1,0.1,512,100,3.647494
3,100,3.943e-06,1,0.01,256,100,6.832226
4,300,6.048e-08,1,0.01,128,150,18.686772
5,600,5.556e-08,1,0.01,128,100,14.367823
6,1000,7.414e-08,1,0.01,256,100,10.336604
7,3000,3.561e-09,1,0.01,512,150,28.694257
8,6000,2.446e-09,1,0.01,512,200,82.725997
9,10000,7.147e-10,1,0.01,1024,300,172.118564


# 3 Run Models on Test Data

In [10]:
np.random.seed(seed=123)
set_seed(1234)

In [11]:
"""
Creates overview with best performing models for each weight and number of hidden layers from grid search
"""

def get_performance_overview(pricer, maturity):
    result_df = pd.read_csv(result_path + 'one_dim_{}_{}_grid_search_all_results.csv'.format(pricer, maturity))
    dfs = []
    for hidden in [1, 2, 3]:
        df = result_df[result_df['Hidden Layers'] == hidden]
        # filter best results for each number of weights
        idx = df.groupby(['Weights'])['Validation MSE'].transform(min) == df['Validation MSE']
        df = df[idx].sort_values('Weights')
        # filter for best run time for each number of weights (only if multiple models have same MSE)
        idx = df.groupby(['Weights'])['Training Time (s)'].transform(min) == df['Training Time (s)']
        df = df[idx].sort_values('Weights')
        dfs.append(df)
    return pd.concat(dfs).reset_index(drop=True)

In [12]:
"""
Trains and evaluates all models with best parameters (from grid search) for a given pricing model and maturity
with early stopping
"""
from time import time

def run_tests(df_overview, X_train, y_train, X_test, y_test, pricer, maturity):
    test_mse = []
    train_epochs = []
    train_time = []
    earlyStop = keras.callbacks.EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)    
    for weights, hidden, lr, batch in zip(df_overview['Weights'], df_overview['Hidden Layers'], 
                                          df_overview['Learning Rate'], df_overview['Batch Size']):
        model = create_model(hidden, float(lr), weights)
        start = time()
        hist = model.fit(X_train, y_train, epochs=5000, batch_size=batch, verbose=0, callbacks=[earlyStop])
        train_time.append(time() - start)
        test_mse.append(model.evaluate(X_test, y_test, verbose=0))
        train_epochs.append(hist.epoch[-1])
    df_overview['Test MSE'] = test_mse
    df_overview['Epochs (Early Stop)'] = train_epochs
    df_overview['Training Time (s)'] = train_time
    df_overview['Test MSE'] = df_overview['Test MSE'].apply(lambda x: '{:.3e}'.format(x))
    df_best = df_overview[df_overview['Rank'] == 1].sort_values('Weights')
    df_best = df_best[['Weights', 'Test MSE', 'Hidden Layers', 'Learning Rate', 'Batch Size',
                           'Epochs (Early Stop)', 'Training Time (s)']]
    df_overview.to_csv(result_path + 'one_dim_{}_{}_test_all_hidden.csv'.format(pricer, maturity), index=False)
    df_best.to_csv(result_path + 'one_dim_{}_{}_test_overview.csv'.format(pricer, maturity), index=False)
    return df_best

In [13]:
"""
Load all grid search overviews
"""

overviews = {}

for pricer in ['merton', 'nig']:
    overviews[pricer] = {}
    for t in T:
        overviews[pricer][t] = get_performance_overview(pricer, t)

In [14]:
"""
Run tests on Merton model data
"""

tests_mert = {}

for t in T:
    tests_mert[t] = run_tests(overviews['merton'][t], X_train_mert, y_train_mert[t], X_test_mert, y_test_mert[t],
                              'merton', t)
    display(tests_mert[t])

,Weights,Test MSE,Hidden Layers,Learning Rate,Batch Size,Epochs (Early Stop),Training Time (s)
0,10,1.270e-04,1,0.1000,256,8,0.539675
1,30,6.408e-06,1,0.1000,128,22,1.159943
2,60,7.239e-03,1,0.0100,128,6,0.635382
3,100,1.689e-06,1,0.0100,512,29,0.639614
4,300,9.652e-08,1,0.0100,128,18,1.188106
5,600,1.283e-07,1,0.0100,128,16,1.158534
6,1000,8.598e-09,1,0.0100,256,36,1.560998
7,3000,2.380e-09,1,0.0100,512,134,6.409905
8,6000,5.027e-09,1,0.0010,512,91,7.227678
9,10000,2.293e-09,1,0.0001,256,215,27.839542


,Weights,Test MSE,Hidden Layers,Learning Rate,Batch Size,Epochs (Early Stop),Training Time (s)
0,10,1.616e-04,1,0.1000,512,19,0.499708
1,30,4.193e-02,1,0.1000,256,7,0.443926
2,60,1.424e-05,1,0.1000,256,39,0.961467
3,100,7.499e-07,1,0.0100,512,90,1.329445
4,300,6.683e-07,1,0.0100,256,24,0.996423
5,600,2.240e-07,1,0.0100,512,19,0.647259
6,1000,3.140e-08,1,0.0010,256,95,2.939142
7,3000,1.344e-08,1,0.0100,512,123,5.917258
8,6000,1.235e-08,1,0.0001,128,125,9.579685
9,10000,9.174e-09,1,0.0001,128,121,23.174075


,Weights,Test MSE,Hidden Layers,Learning Rate,Batch Size,Epochs (Early Stop),Training Time (s)
0,10,2.319e-03,1,0.10,256,13,0.577457
1,30,1.693e-05,1,0.10,512,87,1.125067
2,60,6.077e-05,1,0.10,512,34,0.634269
3,100,6.910e-06,1,0.01,256,63,1.548879
4,300,1.169e-07,1,0.01,128,39,1.989759
5,600,4.306e-08,1,0.01,256,96,2.871320
6,1000,4.388e-08,1,0.01,256,53,1.788242
7,3000,7.947e-09,1,0.01,512,91,4.479994
8,6000,1.477e-08,1,0.01,512,79,6.840712
9,10000,1.141e-10,1,0.01,512,92,11.811423


In [15]:
"""
Run tests on NIG model data
"""

tests_nig = {}

for t in T:
    tests_nig[t] = run_tests(overviews['nig'][t], X_train_nig, y_train_nig[t], X_test_nig, y_test_nig[t],
                              'nig', t)
    display(tests_nig[t])

,Weights,Test MSE,Hidden Layers,Learning Rate,Batch Size,Epochs (Early Stop),Training Time (s)
0,10,3.014e-02,1,0.100,256,8,0.699168
1,30,9.344e-05,1,0.010,128,15,0.939469
2,60,3.250e-06,1,0.100,512,65,1.026305
3,100,1.320e-05,1,0.010,128,18,1.275629
4,300,4.205e-07,1,0.010,512,21,0.600429
5,600,3.027e-08,1,0.010,1024,93,1.525028
6,1000,1.214e-08,1,0.010,256,35,1.379305
7,3000,2.254e-09,1,0.001,512,152,8.018563
8,6000,9.595e-10,1,0.001,512,115,10.823036
9,10000,6.503e-10,1,0.001,1024,172,21.899454


,Weights,Test MSE,Hidden Layers,Learning Rate,Batch Size,Epochs (Early Stop),Training Time (s)
0,10,3.537e-04,1,0.1000,256,13,0.575459
1,30,3.065e-05,1,0.1000,128,15,0.960392
2,60,1.838e-06,1,0.1000,1024,195,1.717422
3,100,1.852e-06,1,0.0100,512,25,0.659238
4,300,2.868e-07,1,0.0100,128,21,1.105256
5,600,4.159e-08,1,0.0100,256,68,1.875051
6,1000,9.227e-09,1,0.0100,512,93,2.001649
7,3000,7.326e-09,1,0.0100,512,129,6.510592
8,6000,2.423e-09,1,0.0100,512,80,6.989283
9,10000,2.615e-08,1,0.0100,512,86,10.474993


,Weights,Test MSE,Hidden Layers,Learning Rate,Batch Size,Epochs (Early Stop),Training Time (s)
0,10,1.514e-04,1,0.10,128,10,0.670236
1,30,6.575e-05,1,0.10,256,16,0.811828
2,60,1.890e-06,1,0.10,512,67,0.939531
3,100,2.005e-06,1,0.01,256,64,1.506021
4,300,5.410e-08,1,0.01,128,45,1.981998
5,600,6.738e-08,1,0.01,128,32,1.646683
6,1000,1.327e-07,1,0.01,256,46,1.514951
7,3000,2.798e-09,1,0.01,512,109,5.605119
8,6000,1.726e-09,1,0.01,512,138,11.741608
9,10000,7.417e-11,1,0.01,1024,204,25.030082
